In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
import matplotlib.pyplot as plt
from datetime import datetime


import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/')
from spicy_snow.IO.user_dates import get_input_dates

# import pickle
# with open('/Users/zachkeskinen/Documents/spicy-snow/SnowEx-Data/Cameron_21-03-19.pkl', 'rb') as f:
#     ds = pickle.load(f)

In [ ]:
pd.to_datetime(datetime(2020, 12, 1))

In [ ]:
get_input_dates('2020-10-01', datetime(2020, 12, 1))

In [ ]:
get_input_dates(date2 = '2020-01-01', date1 = '2021-12-01')